# Stock Prediction Using XGBoost

In this notebook, we'll look at prediction of stock prices usinf XGBoost Algorithm

Importing required Libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from tensorflow.python.framework import random_seed
from matplotlib import pyplot as plt
import xgboost as xgb
from catboost import CatBoostRegressor
import lightgbm as lgb

data_path = 'data/'

## Loading and Data Processing 

Load the data

In [2]:
df_stock = pd.read_csv(data_path+'AAPL.csv')

In [3]:
df_stock.columns = ['Date', 'Stock_Open', 'Stock_High', 'Stock_Low', 'Stock_Close', 'Stock_Adj Close', 'Stock_Volume']

In [4]:
print(df_stock.shape)

(9976, 7)


In [5]:
df_stock.isnull().sum()

Date               0
Stock_Open         1
Stock_High         1
Stock_Low          1
Stock_Close        1
Stock_Adj Close    1
Stock_Volume       1
dtype: int64

In [6]:
df_stock = df_stock.drop(['Date'],axis = 1)

Function for calculation of technical indicators

In [7]:
def technical_indicators(df):
    
    """
    Technical Indicator Calculator Function.
    
    This Function's Output Is A Pandas DataFrame Of Various Techincal Indicators Such As RSI,SMA,EVM,EWMA
    BB And ROC Using Different Time Intervals.
    
    Parameters:
    df (DataFrame) : Pandas DataFrame Of Stock Price
    Returns: 
    new_df (DataFrame) : Pandas DataFrame Of Techincal Indicators
    """
    
    new_df = pd.DataFrame()
    
    dm = ((df['Stock_High'] + df['Stock_Low'])/2) - ((df['Stock_High'].shift(1) + df['Stock_Low'].shift(1))/2)
    br = (df['Stock_Volume'] / 100000000) / ((df['Stock_High'] - df['Stock_Low']))
    EVM = dm / br 
    new_df['EVM_15'] = EVM.rolling(15).mean()

    
    sma_60 = pd.Series.rolling(df['Stock_Close'], window=60, center=False).mean()
    new_df["SMA_60"] = sma_60
    
    sma_200 = pd.Series.rolling(df['Stock_Close'], window=30, center=False).mean()
    new_df["SMA_200"] = sma_200
    
    ewma_50 = df['Stock_Close'].ewm(span = 50, min_periods = 50 - 1).mean()
    new_df["EWMA_50"] = ewma_50
    
    ewma_200 = df['Stock_Close'].ewm(span = 200, min_periods = 200 - 1).mean()
    new_df["EWMA_200"] = ewma_200
    
    sma_5 = pd.Series.rolling(df['Stock_Close'], window=5, center=False).mean()
    std_5 = pd.Series.rolling(df['Stock_Close'], window=5, center=False).std()
    bb_5_upper = sma_5 + (2 * std_5)
    bb_5_lower = sma_5 - (2 * std_5)
    new_df["BB_5_UPPER"] = bb_5_upper
    new_df["BB_5_LOWER"] = bb_5_lower
    new_df["SMA_5"] = sma_5
    
    sma_10 = pd.Series.rolling(df['Stock_Close'], window=10, center=False).mean()
    std_10 = pd.Series.rolling(df['Stock_Close'], window=10, center=False).std()
    bb_10_upper = sma_10 + (2 * std_10)
    bb_10_lower = sma_10 - (2 * std_10)
    new_df["BB_10_UPPER"] = bb_10_upper
    new_df["BB_10_LOWER"] = bb_10_lower
    new_df["SMA_10"] = sma_10
    
    sma_20 = pd.Series.rolling(df['Stock_Close'], window=20, center=False).mean()
    std_20 = pd.Series.rolling(df['Stock_Close'], window=20, center=False).std()
    bb_20_upper = sma_20 + (2 * std_20)
    bb_20_lower = sma_20 - (2 * std_20)
    new_df["BB_20_UPPER"] = bb_20_upper
    new_df["BB_20_LOWER"] = bb_20_lower
    new_df["SMA_20"] = sma_20
    
    roc_5 = df['Stock_Close'][5:]/df['Stock_Close'][:-5].values - 1
    new_df["ROC_5"] = roc_5
    
    roc_10 = df['Stock_Close'][10:]/df['Stock_Close'][:-10].values - 1
    new_df["ROC_10"] = roc_10
    
    roc_20 = df['Stock_Close'][20:]/df['Stock_Close'][:-20].values - 1
    new_df["ROC_20"] = roc_20
    
    delta = df['Stock_Close'].diff()
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    
    up_5 = pd.Series.rolling(up, window=5, center=False).mean()
    down_5 = pd.Series.rolling(down.abs(), window=5, center=False).mean()
    RS_5 = up_5 / down_5
    RSI_5 = 100.0 - (100.0 / (1.0 + RS_5))
    new_df["RSI_5"] = RSI_5
    
    up_10 = pd.Series.rolling(up, window=10, center=False).mean()
    down_10 = pd.Series.rolling(down.abs(), window=10, center=False).mean()
    RS_10 = up_10 / down_10
    RSI_10 = 100.0 - (100.0 / (1.0 + RS_10))
    new_df["RSI_10"] = RSI_10
    
    up_20 = pd.Series.rolling(up, window=20, center=False).mean()
    down_20 = pd.Series.rolling(down.abs(), window=20, center=False).mean()
    RS_20 = up_20 / down_20
    RSI_20 = 100.0 - (100.0 / (1.0 + RS_20))
    new_df["RSI_20"] = RSI_20
    
    return new_df

Preprocessing the data

In [8]:
def process_data(df):
    
    """
    Data Pre-Processing And Cleaning Function.
    
    This Function's Output Is A Pandas DataFrame Which Has No Missing Values And
    And Prices Are Log Scaled Along With Various Technical Indicators.
    
    Parameters:
    df (DataFrame) : Pandas DataFrame Of Stock Price
    Returns: 
    df (DataFrame) : New Pandas DataFrame Of Scaled Features
    """
    
    temp = df
    
    # Fill missing values in data frame, in place.
    df.fillna(method="ffill", inplace=True)
    df.fillna(method="bfill", inplace=True)
    epsilon = 1e-5    
    # log
    df = df.apply(lambda x:np.log(x + epsilon),axis=1)
    temp = technical_indicators(temp)
    
    temp.fillna(method="ffill", inplace=True)
    temp.fillna(method="bfill", inplace=True)
    
    df = pd.concat([df,temp],axis = 1)
    
    
    return df

Generating Time-series data

In [9]:
def get_rolling_data(X,y,train_period,predict_period=1,n_day_later_predict=1):

    """ 
    Generating Timeseries Input And Output Data.  
  
    Parameters: 
    X,y (DataFrame): Features,Labels
    train_period (int): Timesteps For Model
    predict_period (int): Predict On The nth Day Of The End Of The Training Window
    
    Returns: 
    rolling_X (DataFrame): Features
    rolling_y (DataFrame): Labels
    """
    
    assert X.shape[0] == y.shape[0], (
            'X.shape: %s y.shape: %s' % (X.shape, y.shape))
    
    rolling_X, rolling_y = [],[]
    
    for i in range(len(X)-train_period-predict_period-(n_day_later_predict)):

        curr_X=X.iloc[i:i+train_period,:]
        curr_y=y.iloc[i+train_period+n_day_later_predict:i+train_period+predict_period+n_day_later_predict]
        rolling_X.append(curr_X.values.tolist())
        if predict_period == 1:
            rolling_y.append(curr_y.values.tolist()[0])
        else:
            rolling_y.append(curr_y.values.tolist())
        
    rolling_X = np.array(rolling_X)
    rolling_y = np.array(rolling_y)
    return rolling_X, rolling_y

In [10]:
class ModelData():
    
    '''Data Class For Model Train, Predict And Validate.'''
    
    def __init__(self,X,y,seed=None,shuffle=True):
        seed1, seed2 = random_seed.get_seed(seed)
        # If op level seed is not set, use whatever graph level seed is returned
        self._seed = seed1 if seed is None else seed2
        np.random.seed(self._seed)
        
        assert X.shape[0] == y.shape[0], (
            'X.shape: %s y.shape: %s' % (X.shape, y.shape))
        self._num_examples = X.shape[0]
        
        # If shuffle
        if shuffle:
            np.random.seed(self._seed)
            randomList = np.arange(X.shape[0])
            np.random.shuffle(randomList)
            self._X, self._y = X[randomList], y[randomList] 
        
        self._X = X
        self._y = y
        self._epochs_completed = 0
        self._index_in_epoch = 0
        
    
    def train_validate_test_split(self,validate_size=0.10,test_size=0.10):
        
        '''Train, Predict And Validate Splitting Function'''
        
        validate_start = int(self._num_examples*(1-validate_size-test_size)) + 1
        test_start = int(self._num_examples*(1-test_size)) + 1
        if validate_start > len(self._X) or test_start > len(self._X):
            pass
        train_X,train_y = self._X[:validate_start],self._y[:validate_start]
        validate_X, validate_y = self._X[validate_start:test_start],self._y[validate_start:test_start]
        test_X,test_y = self._X[test_start:],self._y[test_start:]
        
        if test_size == 0:
            return ModelData(train_X,train_y,self._seed), ModelData(validate_X,validate_y,self._seed)
        else:
            return ModelData(train_X,train_y,self._seed), ModelData(validate_X,validate_y,self._seed), ModelData(test_X,test_y,self._seed)

    @property
    def X(self):
        return self._X
    
    @property
    def y(self):
        return self._y

Declaring train and predict periods

In [11]:
train_period = 7
predict_period = 1
n_day_later_predict= 7

Splitting the data

In [12]:
price_data = process_data(df_stock)
rolling_X, rolling_y = get_rolling_data(price_data, price_data.loc[:,'Stock_Adj Close'], train_period=train_period, 
                                        predict_period=predict_period,
                                        n_day_later_predict=n_day_later_predict)

model_data = ModelData(rolling_X, rolling_y,seed=666,shuffle=False)
model_train_data, model_validate_data = model_data.train_validate_test_split(validate_size=0.10,test_size=0)

In [13]:
price_data

,Stock_Open,Stock_High,Stock_Low,Stock_Close,Stock_Adj Close,Stock_Volume,EVM_15,SMA_60,SMA_200,EWMA_50,...,SMA_10,BB_20_UPPER,BB_20_LOWER,SMA_20,ROC_5,ROC_10,ROC_20,RSI_5,RSI_10,RSI_20
0,-0.666694,-0.662356,-0.666694,-0.666694,-0.902159,18.579891,0.000132,0.517485,0.559152,0.514863,...,0.518750,0.671497,0.437208,0.554353,-0.017392,0.252173,0.060870,46.153647,75.438569,53.684238
1,-0.715702,-0.715702,-0.720278,-0.720278,-0.955741,17.599045,0.000132,0.517485,0.559152,0.514863,...,0.518750,0.671497,0.437208,0.554353,-0.017392,0.252173,0.060870,46.153647,75.438569,53.684238
2,-0.791565,-0.791565,-0.796503,-0.796503,-1.031965,17.090086,0.000132,0.517485,0.559152,0.514863,...,0.518750,0.671497,0.437208,0.554353,-0.017392,0.252173,0.060870,46.153647,75.438569,53.684238
3,-0.772052,-0.767233,-0.772052,-0.772052,-1.007516,16.888685,0.000132,0.517485,0.559152,0.514863,...,0.518750,0.671497,0.437208,0.554353,-0.017392,0.252173,0.060870,46.153647,75.438569,53.684238
4,-0.743481,-0.738795,-0.743481,-0.743481,-0.978943,16.725816,0.000132,0.517485,0.559152,0.514863,...,0.518750,0.671497,0.437208,0.554353,-0.017392,0.252173,0.060870,46.153647,75.438569,53.684238
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9971,5.886326,5.902579,5.886104,5.899349,5.899349,17.372452,19.772898,313.338499,338.229332,326.842892,...,358.354999,374.892466,321.362527,348.127496,-0.004720,0.036128,0.128224,47.427873,61.981918,67.089862
9972,5.900226,5.906342,5.896907,5.897456,5.897456,17.136376,5.831037,315.032499,339.928331,328.304347,...,359.606998,375.428466,324.725526,350.076996,0.011248,0.035610,0.119925,57.271078,61.749239,66.217454
9973,5.907675,5.914772,5.896164,5.897456,5.897456,17.165779,22.849963,316.777165,341.424331,329.708489,...,360.844995,374.606896,329.726093,352.166495,-0.002001,0.035197,0.129654,48.417833,61.648455,67.796605
9974,5.913503,5.929004,5.913152,5.923855,5.923855,17.205441,34.605745,318.573499,343.324331,331.439529,...,363.257996,376.504705,332.063285,354.283995,0.057178,0.068998,0.127753,96.805543,69.820936,67.949481


In [14]:
price_data.tail(7).values.reshape(-1,1)

array([[ 5.89827965e+00],
       [ 5.90077373e+00],
       [ 5.86652471e+00],
       [ 5.86825121e+00],
       [ 5.86825121e+00],
       [ 1.77534781e+01],
       [ 4.37836080e+01],
       [ 3.09334499e+02],
       [ 3.34765666e+02],
       [ 3.23804383e+02],
       [ 2.82641598e+02],
       [ 3.71024887e+02],
       [ 3.50547111e+02],
       [ 3.60785999e+02],
       [ 3.69730200e+02],
       [ 3.40677796e+02],
       [ 3.55203998e+02],
       [ 3.72756335e+02],
       [ 3.15359659e+02],
       [ 3.44057997e+02],
       [ 1.11803843e-02],
       [ 4.37721887e-02],
       [ 1.12253893e-01],
       [ 5.49783616e+01],
       [ 6.34354196e+01],
       [ 6.54462099e+01],
       [ 5.86717605e+00],
       [ 5.89211378e+00],
       [ 5.86158365e+00],
       [ 5.89103632e+00],
       [ 5.89103632e+00],
       [ 1.73017076e+01],
       [ 3.31491515e+01],
       [ 3.11281999e+02],
       [ 3.36567999e+02],
       [ 3.25293623e+02],
       [ 2.83429045e+02],
       [ 3.71390882e+02],
       [ 3.5

## Models

In [15]:
# process data's dimension
y_train = model_train_data.y[:,np.newaxis]
y_validate = model_validate_data.y[:,np.newaxis]

X_train = model_train_data.X
X_validate = model_validate_data.X

In [16]:
X_train.shape

(8965, 7, 26)

Reshaping the data to fit the model

In [17]:
X_train = X_train.reshape((X_train.shape[0],X_train.shape[1]*X_train.shape[2]))
X_validate = X_validate.reshape((X_validate.shape[0],X_validate.shape[1]*X_validate.shape[2]))

Declaring and training the XGBoost model on data

In [18]:
reg = xgb.XGBRegressor(n_estimators=10000,min_child_weight= 40,learning_rate=0.01,colsample_bytree = 1,subsample = 0.9)
reg.fit(X_train, y_train,eval_set = [(X_validate[:300],y_validate[:300])],early_stopping_rounds = 50) # Change verbose to True if you want to see it train

[17:33:57] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:4.28717
Will train until validation_0-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:4.24808
[2]	validation_0-rmse:4.20941
[3]	validation_0-rmse:4.17096
[4]	validation_0-rmse:4.13304
[5]	validation_0-rmse:4.0952
[6]	validation_0-rmse:4.05812
[7]	validation_0-rmse:4.02119
[8]	validation_0-rmse:3.98372
[9]	validation_0-rmse:3.94773
[10]	validation_0-rmse:3.9121
[11]	validation_0-rmse:3.87567
[12]	validation_0-rmse:3.84073
[13]	validation_0-rmse:3.80505
[14]	validation_0-rmse:3.76956
[15]	validation_0-rmse:3.73545
[16]	validation_0-rmse:3.70089
[17]	validation_0-rmse:3.66777
[18]	validation_0-rmse:3.63365
[19]	validation_0-rmse:3.60107
[20]	validation_0-rmse:3.56889
[21]	validation_0-rmse:3.53705
[22]	validation_0-rmse:3.50542
[23]	validation_0-rmse:3.47295
[24]	validation_0-rmse:3.44065
[25]	validation_0-rmse:3.40966
[26]	validation_0-rmse:3.

[252]	validation_0-rmse:0.538169
[253]	validation_0-rmse:0.534818
[254]	validation_0-rmse:0.531068
[255]	validation_0-rmse:0.527479
[256]	validation_0-rmse:0.523901
[257]	validation_0-rmse:0.520352
[258]	validation_0-rmse:0.51682
[259]	validation_0-rmse:0.513508
[260]	validation_0-rmse:0.510029
[261]	validation_0-rmse:0.506515
[262]	validation_0-rmse:0.503056
[263]	validation_0-rmse:0.499661
[264]	validation_0-rmse:0.496294
[265]	validation_0-rmse:0.493419
[266]	validation_0-rmse:0.490096
[267]	validation_0-rmse:0.486917
[268]	validation_0-rmse:0.483679
[269]	validation_0-rmse:0.480706
[270]	validation_0-rmse:0.477523
[271]	validation_0-rmse:0.474387
[272]	validation_0-rmse:0.471352
[273]	validation_0-rmse:0.468583
[274]	validation_0-rmse:0.465776
[275]	validation_0-rmse:0.462734
[276]	validation_0-rmse:0.459936
[277]	validation_0-rmse:0.457017
[278]	validation_0-rmse:0.454272
[279]	validation_0-rmse:0.451699
[280]	validation_0-rmse:0.448861
[281]	validation_0-rmse:0.44599
[282]	valida

[501]	validation_0-rmse:0.194249
[502]	validation_0-rmse:0.194027
[503]	validation_0-rmse:0.193853
[504]	validation_0-rmse:0.193421
[505]	validation_0-rmse:0.193205
[506]	validation_0-rmse:0.192988
[507]	validation_0-rmse:0.192716
[508]	validation_0-rmse:0.192374
[509]	validation_0-rmse:0.192071
[510]	validation_0-rmse:0.191908
[511]	validation_0-rmse:0.191746
[512]	validation_0-rmse:0.191569
[513]	validation_0-rmse:0.191299
[514]	validation_0-rmse:0.191007
[515]	validation_0-rmse:0.190847
[516]	validation_0-rmse:0.190568
[517]	validation_0-rmse:0.190368
[518]	validation_0-rmse:0.19009
[519]	validation_0-rmse:0.189919
[520]	validation_0-rmse:0.189663
[521]	validation_0-rmse:0.189402
[522]	validation_0-rmse:0.18921
[523]	validation_0-rmse:0.188948
[524]	validation_0-rmse:0.188803
[525]	validation_0-rmse:0.188421
[526]	validation_0-rmse:0.188174
[527]	validation_0-rmse:0.187935
[528]	validation_0-rmse:0.187759
[529]	validation_0-rmse:0.18763
[530]	validation_0-rmse:0.187461
[531]	validat

[750]	validation_0-rmse:0.172263
[751]	validation_0-rmse:0.1722
[752]	validation_0-rmse:0.172178
[753]	validation_0-rmse:0.17217
[754]	validation_0-rmse:0.172157
[755]	validation_0-rmse:0.17215
[756]	validation_0-rmse:0.172057
[757]	validation_0-rmse:0.172025
[758]	validation_0-rmse:0.171956
[759]	validation_0-rmse:0.171872
[760]	validation_0-rmse:0.171852
[761]	validation_0-rmse:0.171849
[762]	validation_0-rmse:0.171842
[763]	validation_0-rmse:0.17187
[764]	validation_0-rmse:0.171862
[765]	validation_0-rmse:0.171847
[766]	validation_0-rmse:0.171702
[767]	validation_0-rmse:0.171693
[768]	validation_0-rmse:0.171694
[769]	validation_0-rmse:0.17173
[770]	validation_0-rmse:0.171735
[771]	validation_0-rmse:0.171652
[772]	validation_0-rmse:0.171703
[773]	validation_0-rmse:0.171695
[774]	validation_0-rmse:0.171685
[775]	validation_0-rmse:0.171667
[776]	validation_0-rmse:0.171539
[777]	validation_0-rmse:0.171468
[778]	validation_0-rmse:0.171454
[779]	validation_0-rmse:0.171454
[780]	validation

[1000]	validation_0-rmse:0.16774
[1001]	validation_0-rmse:0.167737
[1002]	validation_0-rmse:0.167578
[1003]	validation_0-rmse:0.167583
[1004]	validation_0-rmse:0.167579
[1005]	validation_0-rmse:0.167571
[1006]	validation_0-rmse:0.16757
[1007]	validation_0-rmse:0.167419
[1008]	validation_0-rmse:0.167421
[1009]	validation_0-rmse:0.167419
[1010]	validation_0-rmse:0.167422
[1011]	validation_0-rmse:0.167428
[1012]	validation_0-rmse:0.167432
[1013]	validation_0-rmse:0.167421
[1014]	validation_0-rmse:0.167424
[1015]	validation_0-rmse:0.167425
[1016]	validation_0-rmse:0.167414
[1017]	validation_0-rmse:0.167422
[1018]	validation_0-rmse:0.167323
[1019]	validation_0-rmse:0.167348
[1020]	validation_0-rmse:0.167333
[1021]	validation_0-rmse:0.16729
[1022]	validation_0-rmse:0.167294
[1023]	validation_0-rmse:0.167302
[1024]	validation_0-rmse:0.167309
[1025]	validation_0-rmse:0.167312
[1026]	validation_0-rmse:0.16733
[1027]	validation_0-rmse:0.167325
[1028]	validation_0-rmse:0.167324
[1029]	validation_

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.01, max_delta_step=0,
             max_depth=3, min_child_weight=40, missing=None, n_estimators=10000,
             n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
             silent=None, subsample=0.9, verbosity=1)

CatBoost Regressor

In [19]:
model = CatBoostRegressor(iterations=10000,
                          learning_rate=0.005,
                          loss_function = 'RMSE')

model.fit(X_train, y_train,
          eval_set = [(X_validate[:300],y_validate[:300])],
        early_stopping_rounds = 50,
       verbose=True)

0:	learn: 1.8155044	test: 3.7772582	best: 3.7772582 (0)	total: 76.1ms	remaining: 12m 40s
1:	learn: 1.8067196	test: 3.7618582	best: 3.7618582 (1)	total: 87.5ms	remaining: 7m 17s
2:	learn: 1.7979731	test: 3.7442394	best: 3.7442394 (2)	total: 100ms	remaining: 5m 34s
3:	learn: 1.7892706	test: 3.7282203	best: 3.7282203 (3)	total: 113ms	remaining: 4m 43s
4:	learn: 1.7807808	test: 3.7132640	best: 3.7132640 (4)	total: 125ms	remaining: 4m 10s
5:	learn: 1.7722880	test: 3.6980490	best: 3.6980490 (5)	total: 140ms	remaining: 3m 52s
6:	learn: 1.7638142	test: 3.6825948	best: 3.6825948 (6)	total: 152ms	remaining: 3m 37s
7:	learn: 1.7553969	test: 3.6656346	best: 3.6656346 (7)	total: 165ms	remaining: 3m 25s
8:	learn: 1.7469501	test: 3.6494985	best: 3.6494985 (8)	total: 179ms	remaining: 3m 18s
9:	learn: 1.7386149	test: 3.6325327	best: 3.6325327 (9)	total: 192ms	remaining: 3m 11s
10:	learn: 1.7302578	test: 3.6176602	best: 3.6176602 (10)	total: 204ms	remaining: 3m 5s
11:	learn: 1.7218939	test: 3.6006617	be

100:	learn: 1.1232231	test: 2.4773069	best: 2.4773069 (100)	total: 1.3s	remaining: 2m 7s
101:	learn: 1.1179042	test: 2.4659684	best: 2.4659684 (101)	total: 1.32s	remaining: 2m 7s
102:	learn: 1.1125480	test: 2.4556801	best: 2.4556801 (102)	total: 1.33s	remaining: 2m 7s
103:	learn: 1.1072478	test: 2.4447358	best: 2.4447358 (103)	total: 1.34s	remaining: 2m 7s
104:	learn: 1.1019534	test: 2.4354775	best: 2.4354775 (104)	total: 1.35s	remaining: 2m 7s
105:	learn: 1.0967191	test: 2.4251918	best: 2.4251918 (105)	total: 1.36s	remaining: 2m 7s
106:	learn: 1.0915338	test: 2.4148827	best: 2.4148827 (106)	total: 1.38s	remaining: 2m 7s
107:	learn: 1.0863529	test: 2.4050143	best: 2.4050143 (107)	total: 1.39s	remaining: 2m 7s
108:	learn: 1.0811665	test: 2.3952551	best: 2.3952551 (108)	total: 1.4s	remaining: 2m 6s
109:	learn: 1.0760361	test: 2.3855763	best: 2.3855763 (109)	total: 1.41s	remaining: 2m 6s
110:	learn: 1.0709627	test: 2.3759709	best: 2.3759709 (110)	total: 1.43s	remaining: 2m 6s
111:	learn: 

203:	learn: 0.6900809	test: 1.6305281	best: 1.6305281 (203)	total: 2.53s	remaining: 2m 1s
204:	learn: 0.6869130	test: 1.6241133	best: 1.6241133 (204)	total: 2.54s	remaining: 2m 1s
205:	learn: 0.6837026	test: 1.6176443	best: 1.6176443 (205)	total: 2.55s	remaining: 2m 1s
206:	learn: 0.6805124	test: 1.6105846	best: 1.6105846 (206)	total: 2.56s	remaining: 2m 1s
207:	learn: 0.6773164	test: 1.6036248	best: 1.6036248 (207)	total: 2.57s	remaining: 2m 1s
208:	learn: 0.6741646	test: 1.5968358	best: 1.5968358 (208)	total: 2.58s	remaining: 2m 1s
209:	learn: 0.6709846	test: 1.5899465	best: 1.5899465 (209)	total: 2.6s	remaining: 2m 1s
210:	learn: 0.6678690	test: 1.5834124	best: 1.5834124 (210)	total: 2.61s	remaining: 2m
211:	learn: 0.6647586	test: 1.5766355	best: 1.5766355 (211)	total: 2.62s	remaining: 2m
212:	learn: 0.6616560	test: 1.5709035	best: 1.5709035 (212)	total: 2.63s	remaining: 2m
213:	learn: 0.6585670	test: 1.5648826	best: 1.5648826 (213)	total: 2.64s	remaining: 2m
214:	learn: 0.6554972	t

310:	learn: 0.4220914	test: 1.0764401	best: 1.0764401 (310)	total: 3.74s	remaining: 1m 56s
311:	learn: 0.4201805	test: 1.0722848	best: 1.0722848 (311)	total: 3.75s	remaining: 1m 56s
312:	learn: 0.4183116	test: 1.0682974	best: 1.0682974 (312)	total: 3.76s	remaining: 1m 56s
313:	learn: 0.4164242	test: 1.0647066	best: 1.0647066 (313)	total: 3.77s	remaining: 1m 56s
314:	learn: 0.4145418	test: 1.0604584	best: 1.0604584 (314)	total: 3.78s	remaining: 1m 56s
315:	learn: 0.4126882	test: 1.0561712	best: 1.0561712 (315)	total: 3.79s	remaining: 1m 56s
316:	learn: 0.4108706	test: 1.0523268	best: 1.0523268 (316)	total: 3.81s	remaining: 1m 56s
317:	learn: 0.4090336	test: 1.0489459	best: 1.0489459 (317)	total: 3.82s	remaining: 1m 56s
318:	learn: 0.4072054	test: 1.0451012	best: 1.0451012 (318)	total: 3.83s	remaining: 1m 56s
319:	learn: 0.4053985	test: 1.0418551	best: 1.0418551 (319)	total: 3.84s	remaining: 1m 56s
320:	learn: 0.4035848	test: 1.0376517	best: 1.0376517 (320)	total: 3.85s	remaining: 1m 56s

414:	learn: 0.2696883	test: 0.7438234	best: 0.7438234 (414)	total: 4.94s	remaining: 1m 54s
415:	learn: 0.2686041	test: 0.7415748	best: 0.7415748 (415)	total: 4.95s	remaining: 1m 54s
416:	learn: 0.2675131	test: 0.7388633	best: 0.7388633 (416)	total: 4.96s	remaining: 1m 54s
417:	learn: 0.2664266	test: 0.7362047	best: 0.7362047 (417)	total: 4.98s	remaining: 1m 54s
418:	learn: 0.2653675	test: 0.7337893	best: 0.7337893 (418)	total: 4.99s	remaining: 1m 54s
419:	learn: 0.2642840	test: 0.7311537	best: 0.7311537 (419)	total: 5s	remaining: 1m 54s
420:	learn: 0.2632399	test: 0.7286652	best: 0.7286652 (420)	total: 5.01s	remaining: 1m 53s
421:	learn: 0.2621724	test: 0.7260033	best: 0.7260033 (421)	total: 5.02s	remaining: 1m 53s
422:	learn: 0.2611351	test: 0.7236859	best: 0.7236859 (422)	total: 5.03s	remaining: 1m 53s
423:	learn: 0.2600719	test: 0.7211301	best: 0.7211301 (423)	total: 5.04s	remaining: 1m 53s
424:	learn: 0.2590495	test: 0.7189584	best: 0.7189584 (424)	total: 5.05s	remaining: 1m 53s
42

...

1617:	learn: 0.0812490	test: 0.1654872	best: 0.1654633 (1616)	total: 19.6s	remaining: 1m 41s
1618:	learn: 0.0812298	test: 0.1654093	best: 0.1654093 (1618)	total: 19.6s	remaining: 1m 41s
1619:	learn: 0.0812105	test: 0.1653923	best: 0.1653923 (1619)	total: 19.6s	remaining: 1m 41s
1620:	learn: 0.0812028	test: 0.1654188	best: 0.1653923 (1619)	total: 19.6s	remaining: 1m 41s
1621:	learn: 0.0811920	test: 0.1653388	best: 0.1653388 (1621)	total: 19.6s	remaining: 1m 41s
1622:	learn: 0.0811852	test: 0.1653287	best: 0.1653287 (1622)	total: 19.6s	remaining: 1m 41s
1623:	learn: 0.0811805	test: 0.1652518	best: 0.1652518 (1623)	total: 19.6s	remaining: 1m 41s
1624:	learn: 0.0811603	test: 0.1652496	best: 0.1652496 (1624)	total: 19.7s	remaining: 1m 41s
1625:	learn: 0.0811464	test: 0.1652481	best: 0.1652481 (1625)	total: 19.7s	remaining: 1m 41s
1626:	learn: 0.0811356	test: 0.1652806	best: 0.1652481 (1625)	total: 19.7s	remaining: 1m 41s
1627:	learn: 0.0811285	test: 0.1652107	best: 0.1652107 (1627)	total: 1

1715:	learn: 0.0802652	test: 0.1635405	best: 0.1635405 (1715)	total: 20.8s	remaining: 1m 40s
1716:	learn: 0.0802606	test: 0.1635059	best: 0.1635059 (1716)	total: 20.8s	remaining: 1m 40s
1717:	learn: 0.0802475	test: 0.1635062	best: 0.1635059 (1716)	total: 20.8s	remaining: 1m 40s
1718:	learn: 0.0802423	test: 0.1634975	best: 0.1634975 (1718)	total: 20.8s	remaining: 1m 40s
1719:	learn: 0.0802370	test: 0.1634932	best: 0.1634932 (1719)	total: 20.8s	remaining: 1m 40s
1720:	learn: 0.0802244	test: 0.1634854	best: 0.1634854 (1720)	total: 20.9s	remaining: 1m 40s
1721:	learn: 0.0802121	test: 0.1634812	best: 0.1634812 (1721)	total: 20.9s	remaining: 1m 40s
1722:	learn: 0.0802021	test: 0.1635046	best: 0.1634812 (1721)	total: 20.9s	remaining: 1m 40s
1723:	learn: 0.0801948	test: 0.1635125	best: 0.1634812 (1721)	total: 20.9s	remaining: 1m 40s
1724:	learn: 0.0801899	test: 0.1634437	best: 0.1634437 (1724)	total: 20.9s	remaining: 1m 40s
1725:	learn: 0.0801826	test: 0.1634412	best: 0.1634412 (1725)	total: 2

1820:	learn: 0.0793066	test: 0.1624731	best: 0.1624731 (1820)	total: 22s	remaining: 1m 38s
1821:	learn: 0.0793007	test: 0.1624631	best: 0.1624631 (1821)	total: 22s	remaining: 1m 38s
1822:	learn: 0.0792967	test: 0.1623972	best: 0.1623972 (1822)	total: 22s	remaining: 1m 38s
1823:	learn: 0.0792844	test: 0.1623973	best: 0.1623972 (1822)	total: 22s	remaining: 1m 38s
1824:	learn: 0.0792754	test: 0.1623946	best: 0.1623946 (1824)	total: 22s	remaining: 1m 38s
1825:	learn: 0.0792645	test: 0.1623988	best: 0.1623946 (1824)	total: 22.1s	remaining: 1m 38s
1826:	learn: 0.0792538	test: 0.1624037	best: 0.1623946 (1824)	total: 22.1s	remaining: 1m 38s
1827:	learn: 0.0792405	test: 0.1624010	best: 0.1623946 (1824)	total: 22.1s	remaining: 1m 38s
1828:	learn: 0.0792354	test: 0.1623240	best: 0.1623240 (1828)	total: 22.1s	remaining: 1m 38s
1829:	learn: 0.0792288	test: 0.1622795	best: 0.1622795 (1829)	total: 22.1s	remaining: 1m 38s
1830:	learn: 0.0792218	test: 0.1622922	best: 0.1622795 (1829)	total: 22.1s	remai

1910:	learn: 0.0784364	test: 0.1614414	best: 0.1614387 (1907)	total: 23s	remaining: 1m 37s
1911:	learn: 0.0784257	test: 0.1614435	best: 0.1614387 (1907)	total: 23s	remaining: 1m 37s
1912:	learn: 0.0784210	test: 0.1613854	best: 0.1613854 (1912)	total: 23s	remaining: 1m 37s
1913:	learn: 0.0784013	test: 0.1613892	best: 0.1613854 (1912)	total: 23s	remaining: 1m 37s
1914:	learn: 0.0783968	test: 0.1613924	best: 0.1613854 (1912)	total: 23s	remaining: 1m 37s
1915:	learn: 0.0783901	test: 0.1613830	best: 0.1613830 (1915)	total: 23.1s	remaining: 1m 37s
1916:	learn: 0.0783794	test: 0.1613824	best: 0.1613824 (1916)	total: 23.1s	remaining: 1m 37s
1917:	learn: 0.0783736	test: 0.1613829	best: 0.1613824 (1916)	total: 23.1s	remaining: 1m 37s
1918:	learn: 0.0783622	test: 0.1614073	best: 0.1613824 (1916)	total: 23.1s	remaining: 1m 37s
1919:	learn: 0.0783547	test: 0.1614076	best: 0.1613824 (1916)	total: 23.1s	remaining: 1m 37s
1920:	learn: 0.0783453	test: 0.1614387	best: 0.1613824 (1916)	total: 23.1s	remai

2015:	learn: 0.0774729	test: 0.1608852	best: 0.1608730 (1997)	total: 24.2s	remaining: 1m 35s
2016:	learn: 0.0774658	test: 0.1608827	best: 0.1608730 (1997)	total: 24.2s	remaining: 1m 35s
2017:	learn: 0.0774612	test: 0.1609116	best: 0.1608730 (1997)	total: 24.2s	remaining: 1m 35s
2018:	learn: 0.0774549	test: 0.1609507	best: 0.1608730 (1997)	total: 24.2s	remaining: 1m 35s
2019:	learn: 0.0774457	test: 0.1609513	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2020:	learn: 0.0774361	test: 0.1609002	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2021:	learn: 0.0774291	test: 0.1609006	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2022:	learn: 0.0774160	test: 0.1609014	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2023:	learn: 0.0774056	test: 0.1608998	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2024:	learn: 0.0773954	test: 0.1609011	best: 0.1608730 (1997)	total: 24.3s	remaining: 1m 35s
2025:	learn: 0.0773916	test: 0.1608941	best: 0.1608730 (1997)	total: 2

#### Ensemble the 2 models

In [20]:
y_val_pred = (reg.predict(X_validate) + model.predict(X_validate)) / 2
y_train_pred = (reg.predict(X_train) + model.predict(X_train)) / 2

In [21]:
print(np.exp(reg.predict(price_data.tail(7).values.reshape(1,-1))))

[115.87135]


In [22]:
y_val_pred = np.exp(y_val_pred)
y_val_org = np.exp(y_validate) 

In [23]:
y_train_pred = np.exp(y_train_pred)
y_train_org = np.exp(y_train)

In [24]:
from sklearn.metrics import r2_score,mean_absolute_error

In [25]:
print('MAE on Val Data: '+ str(mean_absolute_error(y_val_org,y_val_pred)))
print('MAE on Train Data: '+ str(mean_absolute_error(y_train_org,y_train_pred)))

MAE on Val Data: 72.90703004004116
MAE on Train Data: 0.5271772738421181


In [26]:
print('R-score on Val Data: '+ str(r2_score(y_val_org[:],y_val_pred[:])))
print('R-score on train Data: '+ str(r2_score(y_train_org,y_train_pred)))

R-score on Val Data: -1.3750692705052256
R-score on train Data: 0.9980204411868966
